# Codelab PyTorch (beginner)
## Organizzato da [Italian Association for Machine Learning](http://iaml.it) e [Sourcesense](https://www.sourcesense.com)

### Preparazione della macchine virtuale

Si veda il sito ufficiale per le varie opzioni di install: http://pytorch.org

In [0]:
# Installazione di PyTorch (versione in preview 1.0.0)
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu90/torch_nightly.html

In [0]:
# Installazione di scikit-learn e matplotlib
!pip install scikit-learn matplotlib

### Parte 1: Concetti base di PyTorch

In [0]:
import torch

In [0]:
# Creare un tensore in NumPy
import numpy as np
x = np.zeros((3, 2))
print(x)

In [0]:
# Creare un tensore simile in PyTorch
y = torch.ones((3, 2))
print(y.double())

In [0]:
# Shape del tensore
y.shape

In [0]:
# Indicizzazione
y[0, 0]

In [0]:
# Interoperabilità NumPy / PyTorch
# Notare come la rappresentazione sia condivisa!
xx = torch.from_numpy(x)
xx += 1
x

In [0]:
# Interoperabilità PyTorch / NumPy
y.numpy()

In [0]:
# Nuova sintassi in > 0.4.0 per ottenere un singolo scalare
y[0,0].item()

In [0]:
# Statistiche sull'intero tensore
x.mean(axis=0)

### Parte 2: meccanismi di autodifferenziazione

In [0]:
# Definire un tensore di cui tracciare le operazioni
# Nota: nelle versioni < 0.4.0, questo si otteneva con un oggetto di tipo "Variable"
v = torch.rand(4, requires_grad=True)

In [0]:
# Ottenere i dati del tensore
v.data

In [0]:
# 'grad_fn' mantiene informazioni sulle varie operazioni con cui sono stati costruiti i tensori
v.grad_fn

In [0]:
# Effettuiamo qualche operazione e vediamo il tracciamento
z = torch.sum(torch.sqrt(v + 3))
z.grad_fn

In [0]:
# Operazione di autodifferenziazione
z.backward()

In [0]:
# Il gradiente viene salvato all'interno dei tensori stessi
v.grad

In [0]:
# Per ottenere la rappresentazione NumPy è necessario chiamare 'detach' per ottenere una 'copia sicura' dei dati (senza tracciamento)
v.detach().numpy()

In [0]:
# Sintassi per operazioni che non richiedono tracciamento
with torch.no_grad():
  z = v + 3.0

### Parte 3: un classificatore in PyTorch

In [0]:
# Importiamo un dataset di classificazione (Iris)
from sklearn import datasets
data = datasets.load_iris()

In [0]:
# In sklearn.datasets, i dataset sono in forma di dizionario
data.keys()

In [0]:
# Estraiamo i dati utili per l'allenamento
X = data['data']
y = data['target']

In [0]:
X.shape

In [0]:
y

In [0]:
# Dividiamo i dati in una parte di training ed una parte di test
from sklearn import model_selection
Xtrain, Xtest, ytrain, ytest =\
      model_selection.train_test_split(X, y, stratify=y)

In [0]:
# Esempio di regressione logistica implementata 'a mano'
w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(3, requires_grad=True)
def linear(x):
  return x.mm(w) + b

In [0]:
# Esempio di rete neurale implementata con le classi ad alto livello di PyTorch
class OurModel(torch.nn.Module):
  
  def __init__(self):
    super(OurModel, self).__init__()
    self.lin = torch.nn.Linear(4, 15)
    self.nonlin = torch.nn.ReLU()
    self.lin2 = torch.nn.Linear(15, 3)
    
  def forward(self, x):
    return self.lin2(self.nonlin(self.lin(x)))

In [0]:
# Iniziallizazione del modello
nn = OurModel()

In [0]:
# Esempio di predizione del modello
nn(torch.from_numpy(Xtrain[0:3]).float())

In [0]:
# Per ottenere una lista di parametri del modello
# Nota 1: 'Parameter' è un oggetto che denota un tensore di parametri del modello (practicamente equivalente ad un tensore con requires_grad=True)
# Nota 2: nn.parameters() ritorna un generatore
# Nota 3: esiste anche una versione 'named_parameters' per ottenere i 'nomi' dei vari parametri
[p for p in nn.parameters()]

In [0]:
# Definizione della funzione costo
crossentropy = torch.nn.CrossEntropyLoss()
def loss(x, y):
  ypred = nn(x)
  return crossentropy(ypred, y)

In [0]:
# Definizione dell'algoritmo di ottimizzazione
opt = torch.optim.Adam(params=nn.parameters())

In [0]:
# Fase di ottimizzazione
iters = 1000

Xtrain_t = torch.from_numpy(Xtrain).float()
ytrain_t = torch.from_numpy(ytrain)

all_losses = np.zeros(iters)

for i in range(iters):
  
  l = loss(Xtrain_t, ytrain_t)
  all_losses[i] = l.detach().item()
  
  l.backward()
  
  opt.step()
  
  opt.zero_grad()

In [0]:
# Plot della loss del modello durante l'ottimizzazione
import matplotlib.pyplot as plt
plt.plot(all_losses)
plt.show()

In [0]:
# Calcolo dell'accuratezza
ypred = nn(torch.from_numpy(Xtest).float()).detach().numpy()
np.mean(np.argmax(ypred, axis=1) == ytest)

In [0]:
# Esempio di come spostare tutti i conti sulla GPU
Xtrain_t.to(torch.device('cuda'))
ytrain_t.to(torch.device('cuda'))
nn.to(torch.device('cuda'))